In [1]:
# -*- coding: UTF-8 -*-
"""Prediction of survival at the Titanic with GradientBoosting classifier"""
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, GridSearchCV, train_test_split, cross_val_score, RepeatedKFold, RepeatedStratifiedKFold, cross_validate
from sklearn import metrics
from sklearn import preprocessing
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier,  HistGradientBoostingClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier

In [2]:
def exec_time(start:float, end:float) -> float:
    """Measures code runtime, returns value in milliseconds rounded to 4 decimal places."""
    diff_time = (end - start) * 1000
    return round(diff_time, 4)

In [3]:
def get_metrics(y_test, y_pred) -> map:
    """Calculates the main model metrics - accuracy, balanced-Accuracy, recall, precision`, f1_score and returns them as a map object"""
    accuracy = metrics.accuracy_score(y_test, y_pred)
    balanced_accuracy = metrics.balanced_accuracy_score(y_test, y_pred)
    precision = metrics.precision_score(y_test, y_pred)
    recall = metrics.recall_score(y_test, y_pred)
    f_1 = metrics.f1_score(y_test, y_pred)
    roc_auc = metrics.roc_auc_score(y_test, y_pred)
    metrics = map(lambda element: round(element, 4), (accuracy, balanced_accuracy, precision, recall, f_1, roc_auc))
    return metrics

In [14]:
# Define test classification dataset
X, y = make_classification(n_samples=20000, n_features=20, n_informative=15, n_redundant=2, random_state=12)
# summarize the dataset
print(X.shape, y.shape)
# Split for train/test data as 80% and 20 %
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12)
X, y

(20000, 20) (20000,)


(array([[-0.26738173,  1.85662962,  2.76054246, ...,  1.44769036,
          0.66436077, -0.71182529],
        [-4.7460244 , -2.88790148,  1.00161548, ...,  3.6427129 ,
         -3.69951663, -0.79651019],
        [ 3.12251606, -2.51194452,  0.77282303, ...,  1.29561269,
          0.95566055,  0.68894349],
        ...,
        [ 0.22319313, -2.69164195, -3.39125388, ...,  4.91436425,
          0.25447676, -2.33166532],
        [ 1.48123102, -2.05414838, -0.73597419, ...,  4.81923953,
         -1.09124714, -2.04247616],
        [ 3.7372374 , -7.33252479,  2.55822322, ...,  1.81333293,
         -2.01652742, -1.89884291]]),
 array([1, 1, 0, ..., 0, 0, 1]))

In [11]:
# instance the classification models
rf = RandomForestClassifier()
gb = GradientBoostingClassifier()
hist_gb = HistGradientBoostingClassifier()
xgb = XGBClassifier()
lgb = LGBMClassifier()
catgb = CatBoostClassifier(verbose=0, n_estimators=200)
mlp = MLPClassifier(max_iter=1000)

models = (rf, gb, hist_gb, xgb, lgb, catgb, mlp)

models

(RandomForestClassifier(),
 GradientBoostingClassifier(),
 HistGradientBoostingClassifier(),
 XGBClassifier(base_score=None, booster=None, callbacks=None,
               colsample_bylevel=None, colsample_bynode=None,
               colsample_bytree=None, early_stopping_rounds=None,
               enable_categorical=False, eval_metric=None, gamma=None,
               gpu_id=None, grow_policy=None, importance_type=None,
               interaction_constraints=None, learning_rate=None, max_bin=None,
               max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
               max_leaves=None, min_child_weight=None, missing=nan,
               monotone_constraints=None, n_estimators=100, n_jobs=None,
               num_parallel_tree=None, predictor=None, random_state=None,
               reg_alpha=None, reg_lambda=None, ...),
 LGBMClassifier(),
 MLPClassifier(max_iter=1000))

In [6]:
def make_cross_validation(estimator, X_train, y_train, cv=5):
    scorings = ('accuracy', 'balanced_accuracy', 'f1', 'precision', 'recall',  'roc_auc')
    scores = cross_validate(estimator, X_train, y_train, cv=cv, n_jobs=-1, scoring=scorings)
    scores.items()
    final_metrics = dict()
    for key, item in sorted(scores.items()):
        # print(f'{key} is {item.mean().round(4)}')
        final_metrics[key] = item.mean().round(4)
    return final_metrics

In [16]:
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=12)
data = []
for mdl in models:
    report_dict = make_cross_validation(mdl, X_train, y_train, cv=cv)
    data.append(report_dict)
df_report = pd.DataFrame(data, index=models)
df_report

KeyboardInterrupt: 

In [7]:
def make_fitting(estimator, X_train, y_train):
    estimator.fit(X_train, y_train)
    return estimator

In [8]:
def make_prediction(estimator, X_test):
    y_pred = estimator.predict(X_test)
    return y_pred

In [15]:
for mdl in models:
    mdl.fit(X_train, y_train)
    # estimator = make_fitting(mdl, X_train, y_train)
    # y_pred = make_prediction(estimator, X_test)
    y_pred = mdl.predict(X_test)
    accaracy = metrics.accuracy_score(y_test, y_pred)
    bal_accuracy = metrics.balanced_accuracy_score(y_test, y_pred)
    precision = metrics.precision_score(y_test, y_pred)
    recall = metrics.recall_score(y_test, y_pred)
    f1_score = metrics.f1_score(y_test, y_pred)
    roc_auc = metrics.roc_auc_score(y_test, y_pred)
    # mult_nb_balanced_accuracy, mult_nb_precision, mult_nb_recall, mult_nb_f1_score = get_metrics(y_test, y_pred)
    print(f'\nTesting the best model: {mdl}')
    print(f"Accuracy: {accaracy.round(4)}, Balanced Accuracy: {bal_accuracy.round(4)}, Precision {precision.round(4)},  Recall {recall.round(4)}, F1_score {f1_score.round(4)}")


Testing the best model: RandomForestClassifier()
Accuracy: 0.955, Balanced Accuracy: 0.9549, Precision 0.9534,  Recall 0.9534, F1_score 0.9534

Testing the best model: GradientBoostingClassifier()
Accuracy: 0.919, Balanced Accuracy: 0.919, Precision 0.9149,  Recall 0.9177, F1_score 0.9163

Testing the best model: HistGradientBoostingClassifier()
Accuracy: 0.9685, Balanced Accuracy: 0.9684, Precision 0.9689,  Recall 0.9659, F1_score 0.9674

Testing the best model: XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, mo